In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip "/content/gdrive/MyDrive/AI/fer2013.zip" -d "/content/gdrive/MyDrive/AI"

### Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model


### Train Data Information

In [ ]:
def dataset_info(datapath):
  print(datapath.split("/")[-2] + ": ")
  sum = 0
  for emotion in os.listdir(datapath):
    length = len(os.listdir(datapath + emotion))
    sum += length
    print(f"{str(length)} {emotion} images")
  print(f"Total: {sum}\n")

In [ ]:
dataset_info("/content/gdrive/MyDrive/AI/fer2013/train/")
dataset_info("/content/gdrive/MyDrive/AI/fer2013/validation/")
dataset_info("/content/gdrive/MyDrive/AI/fer2013/test/")

### Data Augmentation and Generation

In [ ]:
BS = 128

def get_datagen(dataset, aug=False):
    if aug:
        datagen = ImageDataGenerator(rescale=1./255, featurewise_center=False, featurewise_std_normalization=False,
                            rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
                            zoom_range=0.1, horizontal_flip=True)
    else:
        datagen = ImageDataGenerator(rescale=1./255)

    return datagen.flow_from_directory(dataset, target_size=(48, 48), color_mode='grayscale',
            shuffle = True, class_mode='categorical', batch_size=BS)

In [ ]:
train_gen = get_datagen('/content/gdrive/MyDrive/AI/fer2013/train', True)
val_gen = get_datagen('/content/gdrive/MyDrive/AI/fer2013/validation')
test_gen = get_datagen('/content/gdrive/MyDrive/AI/fer2013/test')

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


### Convolutional Neural Network

In [ ]:
model = Sequential()
model.add(BatchNormalization(input_shape=(48,48,1)))
model.add(Conv2D(32, (3, 3), activation='relu',padding='same', input_shape=(48,48,1),name="conv1"))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool1"))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu',padding='same',name="conv2"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool2"))         
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv3"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool3"))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv4"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool4"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1024, activation='relu',name='fc1'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(7, activation='softmax',name='fcsoftmax'))

sgd = SGD(learning_rate=0.01,momentum=0.9, decay=0.0001, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

### Training Neural Network

In [ ]:
epochs = 50
steps_per_epoch = train_gen.n // train_gen.batch_size
validation_steps = val_gen.n // val_gen.batch_size

model_path = "/content/gdrive/MyDrive/AI/models/"

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',mode='max',factor=0.5, patience=10, min_lr=0.00001, verbose=1)
checkpoint = ModelCheckpoint(model_path+"model_weights.h5", monitor='val_accuracy', verbose=1, save_weights_only=True,
                             save_best_only=True, mode='max')
callbacks_list = [reduce_lr,checkpoint]

history = model.fit(x = train_gen, validation_data = val_gen, shuffle=True,
                              epochs=50, callbacks = callbacks_list, use_multiprocessing=False,)

Epoch 1/50
225/225 [==============================] - 19s 86ms/step - loss: 1.5867 - accuracy: 0.3912 - val_loss: 1.4609 - val_accuracy: 0.4609

Epoch 00001: val_accuracy improved from -inf to 0.46085, saving model to models/model_weights.h5
Epoch 2/50
225/225 [==============================] - 19s 84ms/step - loss: 1.5232 - accuracy: 0.4184 - val_loss: 1.5110 - val_accuracy: 0.4653

Epoch 00002: val_accuracy improved from 0.46085 to 0.46531, saving model to models/model_weights.h5
Epoch 3/50
225/225 [==============================] - 19s 84ms/step - loss: 1.4806 - accuracy: 0.4331 - val_loss: 1.3147 - val_accuracy: 0.5071

Epoch 00003: val_accuracy improved from 0.46531 to 0.50711, saving model to models/model_weights.h5
Epoch 4/50
225/225 [==============================] - 19s 83ms/step - loss: 1.4454 - accuracy: 0.4490 - val_loss: 1.3235 - val_accuracy: 0.5032

Epoch 00004: val_accuracy did not improve from 0.50711
Epoch 5/50
225/225 [==============================] - 19s 83ms/step 

### Save Model Architecture

In [ ]:
model_json = model.to_json()
with open(model_path+"model.json","w") as json_file:
    json_file.write(model_json)

In [ ]:
results_test = model.evaluate_generator(test_gen, 3560 // BS)
print('test loss, test acc:', results_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


test loss, test acc: [0.9906769394874573, 0.6385995149612427]
